In [107]:
%%capture --no-stderr
%pip install -U langchain_openai langgraph langchain_core langchain_community pytrends requests beautifulsoup4 selenium webdriver_manager chromedriver-autoinstaller

In [19]:
import os, getpass

def _set_env(var: str):
    # Check if the variable is set in the OS environment
    env_value = os.environ.get(var)
    if not env_value:
        # If not set, prompt the user for input
        env_value = getpass.getpass(f"{var}: ")
    
    # Set the environment variable for the current process
    os.environ[var] = env_value

_set_env("OPENAI_API_KEY")

In [20]:
from typing import List
from pydantic import BaseModel, Field

class GeneratedWords(BaseModel):
    words: List[str] = Field(..., description="List of the words beloging to a specific context")


In [21]:
from langchain_openai import ChatOpenAI

llm = ChatOpenAI(model="gpt-4o-mini", temperature=0.0, max_tokens=12000).with_retry(
    stop_after_attempt=10
)

# Using with_structured_output feature to parse the output
# to the pydantic models previous defined
# with_retry -> https://python.langchain.com/api_reference/core/runnables/langchain_core.runnables.base.Runnable.html#langchain_core.runnables.base.Runnable.with_retry
llm_w_structured_output = (
    ChatOpenAI(model="gpt-4o-mini", temperature=0.0)
    .with_structured_output(GeneratedWords)
    .with_retry(stop_after_attempt=10)  
)

In [65]:
from langchain_core.prompts import ChatPromptTemplate

context = """
            🚀 Contexto
            Hoje, o WhatsApp é um dos principais canais de vendas para pequenas e médias empresas (PMEs). No entanto, muitas delas enfrentam dificuldades em gerenciar mensagens, responder clientes a tempo e transformar conversas em vendas. O atendimento manual consome tempo e, muitas vezes, resulta em oportunidades perdidas.

                ❌ O Problema das PMEs
                📉 Perda de vendas por demora nas respostas ou atendimentos fora do horário.
                ⏳ Falta de tempo para responder manualmente a cada cliente.
                💬 Dificuldade em manter leads engajados e conduzi-los até a compra.
                🔗 Falta de integração com ferramentas que otimizam o processo de vendas.

            ✅ Nossa Solução: O Vendedor IA
            Criamos um Vendedor IA para WhatsApp, que automatiza o atendimento, responde clientes 24/7 e impulsiona as vendas sem necessidade de intervenção manual. Ele aprende sobre a marca, os produtos e os valores da empresa para oferecer um atendimento mais personalizado e eficiente.

                🎯 Principais Benefícios
                🤖 Atendimento Automático 24/7 – Nunca mais perca vendas por falta de resposta.
                ⚡ Respostas rápidas e inteligentes – Mantém clientes engajados no momento certo.
                🛠 Fácil de configurar – Sem necessidade de programação, pronto em minutos.
                📊 Dashboard com métricas – Acompanhe interações, leads e conversões.
                🔗 Integração com CRM – Gerencie leads e otimize o processo de vendas.
                📲 Totalmente conectado ao WhatsApp Business – Funcionamento fluido no principal canal de vendas das PMEs.

            🤖 Mais que um Chatbot: Seu Vendedor IA
                Muita gente já ouviu falar em chatbots, mas nosso Vendedor IA vai além!

                Os chatbots tradicionais apenas respondem perguntas de forma automática. Já o Vendedor IA foi criado para entender sua marca, aprender sobre seus produtos e vender de verdade no WhatsApp.

                O que isso significa na prática?
                🚀 Ele não só responde – ele engaja e converte.
                🧠 Aprende sobre sua empresa para representar seu negócio de forma personalizada.
                📊 Fornece métricas e insights, ajudando você a vender mais.
                🔗 Integra-se com WhatsApp Business e CRM para um fluxo de vendas completo.

                💡 Enquanto um chatbot apenas atende, nosso Vendedor IA trabalha ativamente para aumentar suas vendas!
        """

prompt = ChatPromptTemplate(
    [
        (
            "system",
            """Atue como um especialista em SEO e análise de tendências de busca. Gere uma lista de **10 palavras-chave (palavras únicas)** relacionadas ao seguinte contexto:

                ------------------------------------------------------------------
                {context}
                ------------------------------------------------------------------

                🔹 **Requisitos:**  
                1. As palavras devem ser altamente relevantes para pesquisas no Google.  
                2. Inclua tanto **termos amplos** quanto **termos de cauda longa** para melhor identificação de tendências.  
                3. Priorize palavras com potencial para aparecer em **Featured Snippets** e influenciar a seção **People Also Ask (PAA)**.  
                4. Não inclua perguntas, apenas **termos principais** que possam ser usados para otimização de conteúdo.  
 
            """,
        ),
    ]
)

In [122]:
prompt.invoke(input={ 'context': context})

ChatPromptValue(messages=[SystemMessage(content='Atue como um especialista em SEO e análise de tendências de busca. Gere uma lista de **10 palavras-chave (palavras únicas)** relacionadas ao seguinte contexto:\n\n                ------------------------------------------------------------------\n                \n            🚀 Contexto\n            Hoje, o WhatsApp é um dos principais canais de vendas para pequenas e médias empresas (PMEs). No entanto, muitas delas enfrentam dificuldades em gerenciar mensagens, responder clientes a tempo e transformar conversas em vendas. O atendimento manual consome tempo e, muitas vezes, resulta em oportunidades perdidas.\n\n                ❌ O Problema das PMEs\n                📉 Perda de vendas por demora nas respostas ou atendimentos fora do horário.\n                ⏳ Falta de tempo para responder manualmente a cada cliente.\n                💬 Dificuldade em manter leads engajados e conduzi-los até a compra.\n                🔗 Falta de integração 

In [66]:
chain = prompt | llm_w_structured_output

relevant_words = chain.invoke(input={ 'context': context})

In [29]:
from pytrends.request import TrendReq

pytrends = TrendReq(hl='pt-BR', tz=360, timeout=(10,25))#, proxies=['https://72.10.160.172:11723'])

In [67]:
relevant_words.words

['WhatsApp',
 'atendimento',
 'automação',
 'vendas',
 'chatbot',
 'integração',
 'leads',
 'personalização',
 'CRM',
 'PMEs']

In [55]:
[str(w) for w in relevant_words.words[:2]]

['atendimento automático', 'WhatsApp Business']

In [68]:
# relevant_words.words[:1]
word_stats = []
for w in relevant_words.words:
    print(w)
    pytrends.build_payload(kw_list=[w], timeframe='2025-01-01 2025-03-08', geo='BR-SP')
    word_stats.append(pytrends.interest_over_time())


WhatsApp
atendimento
automação
vendas
chatbot
integração
leads
personalização
CRM
PMEs


In [73]:
word_stats[k].mean()[0]

/tmp/ipykernel_31938/3195702251.py:1: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  word_stats[k].mean()[0]


np.float64(11.373134328358208)

In [75]:
for k, w in enumerate(relevant_words.words):
    # print(w)
    # print(word_stats[k].mean())
    # print(word_stats[k])
    print(f"{w}: {word_stats[k].mean().iloc[0]}")

WhatsApp: 71.22388059701493
atendimento: 72.94029850746269
automação: 61.92537313432836
vendas: 78.6268656716418
chatbot: 51.1044776119403
integração: 35.776119402985074
leads: 55.71641791044776
personalização: 29.52238805970149
CRM: 76.32835820895522
PMEs: 11.373134328358208


In [35]:
pytrends.build_payload(kw_list=['pizza', 'vendas'], timeframe=['2022-09-04 2022-09-10', '2022-09-18 2022-09-24'], geo='BR-SP')
pytrends.multirange_interest_over_time()


,[0] pizza date,[0] pizza value,[1] vendas date,[1] vendas value
0,Average,67,Average,27
1,4 de set. de 2022,88,18 de set. de 2022,17
2,5 de set. de 2022,45,19 de set. de 2022,30
3,6 de set. de 2022,53,20 de set. de 2022,33
4,7 de set. de 2022,76,21 de set. de 2022,31
5,8 de set. de 2022,47,22 de set. de 2022,32
6,9 de set. de 2022,63,23 de set. de 2022,27
7,10 de set. de 2022,100,24 de set. de 2022,20


In [14]:
pytrends.multirange_interest_over_time()


""


In [14]:
pytrends.interest_by_region(resolution='CITY', inc_low_vol=True, inc_geo_code=False)


,pizza,bagel
geoName,,
Abbotsford,0,0
Akron,100,0
Albany,100,0
Allentown,0,0
Avalon,100,0
...,...,...
Windsor,0,0
Winnipeg,100,0
Winter Haven,100,0


In [ ]:
import requests

def get_google_autocomplete_suggestions(query):
    url = f"https://suggestqueries.google.com/complete/search?client=firefox&hl=pt-BR&gl=br&q={query}"
    response = requests.get(url)
    print(response.json())
    if response.status_code == 200:
        return response.json()[1]  # As sugestões estão na segunda posição da lista retornada
    return []

query = "Como automatizar o atendimento no WhatsApp"
suggestions = get_google_autocomplete_suggestions(query)
print(suggestions)

['Como automatizar o atendimento no WhatsApp', ['como automatizar o atendimento no whatsapp', 'como automatizar atendimento no whatsapp business', 'como automatizar o atendimento via whatsapp', 'como automatizar atendimento pelo whatsapp'], [], {'google:suggestsubtypes': [[512], [22, 30], [22, 30], [22, 30]]}]
['como automatizar o atendimento no whatsapp', 'como automatizar atendimento no whatsapp business', 'como automatizar o atendimento via whatsapp', 'como automatizar atendimento pelo whatsapp']


In [81]:
import requests
from bs4 import BeautifulSoup
def google_search(query):
    query = query.replace(' ', '+')
    url = f"https://www.google.com/search?q={query}"
    headers = {
        "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.124 Safari/537.36"
    }   
    response = requests.get(url, headers=headers)
    if response.status_code == 200:
        print("Connection successful!")
        return BeautifulSoup(response.text, "html.parser")
    else:
        print(f"Error: Unable to fetch the search results. Status code: {response.status_code}")
        return None

In [82]:
def extract_questions(soup):
    questions = []
    if soup:
        for question in soup.select('span.CSkcDe'):
            questions.append(question.get_text())
        return questions

In [83]:
query = "como automatizar o whatsapp"
soup = google_search(query)
questions = extract_questions(soup)


Connection successful!


In [109]:
import chromedriver_autoinstaller
chromedriver_autoinstaller.install()


'/home/vscode/.local/lib/python3.11/site-packages/chromedriver_autoinstaller/134/chromedriver'

In [117]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.service import Service
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.chrome.options import Options

import time

def get_paa_questions(query):

    options = Options()
    options.add_argument("--headless")  # Executa sem abrir a janela (modo headless)
    options.add_argument("--no-sandbox")  # Recomendado para ambientes Linux
    options.add_argument("--disable-dev-shm-usage")  # Evita problemas de memória compartilhada


    # Definir um User-Agent realista
    options.add_argument(
        "user-agent=Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/119.0.0.0 Safari/537.36"
    )

    service = Service(ChromeDriverManager().install())
    driver = webdriver.Chrome(service=service, options=options)
    # Configurar WebDriver do Chrome
    # service = Service(ChromeDriverManager().install())
    # driver = webdriver.Chrome(service=service)

    # Abrir Google e pesquisar
    google_search_url = f"https://www.google.com/search?q={query}"
    driver.get(google_search_url)
    time.sleep(3)

    # Encontrar perguntas do PAA
    paa_questions = []
    questions = driver.find_elements(By.XPATH, "//div[@jsname='yEVEwb']/span")

    for question in questions:
        paa_questions.append(question.text)
        question.click()
        time.sleep(1)  # Pequena pausa para carregar mais perguntas

    driver.quit()
    return paa_questions

query = "como melhorar o SEO"
paa_results = get_paa_questions(query)

print("Perguntas do PAA:")
for question in paa_results:
    print(f"- {question}")


Perguntas do PAA:


In [ ]:
options = Options()
options.add_argument("--headless")  # Executa sem abrir a janela (modo headless)
options.add_argument("--no-sandbox")  # Recomendado para ambientes Linux
options.add_argument("--disable-dev-shm-usage")  # Evita problemas de memória compartilhada


# Definir um User-Agent realista
options.add_argument(
    "user-agent=Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/119.0.0.0 Safari/537.36"
)

service = Service(ChromeDriverManager().install())
driver = webdriver.Chrome(service=service, options=options)
query = "como melhorar o SEO"
# Abrir Google e pesquisar
google_search_url = f"https://www.google.com/search?q={query}"
driver.get(google_search_url)

In [90]:
query

'como automatizar o whatsapp'

In [119]:
driver.page_source

'<html><head><meta http-equiv="origin-trial" content="A/kargTFyk8MR5ueravczef/wIlTkbVk1qXQesp39nV+xNECPdLBVeYffxrM8TmZT6RArWGQVCJ0LRivD7glcAUAAACQeyJvcmlnaW4iOiJodHRwczovL2dvb2dsZS5jb206NDQzIiwiZmVhdHVyZSI6IkRpc2FibGVUaGlyZFBhcnR5U3RvcmFnZVBhcnRpdGlvbmluZzIiLCJleHBpcnkiOjE3NDIzNDIzOTksImlzU3ViZG9tYWluIjp0cnVlLCJpc1RoaXJkUGFydHkiOnRydWV9"><meta http-equiv="content-type" content="text/html; charset=utf-8"><meta name="viewport" content="initial-scale=1"><title>https://www.google.com/search?q=como%20melhorar%20o%20SEO&amp;sei=JBnOZ6KJN_SU5OUPst7YyAM</title></head>\n<body style="font-family: arial, sans-serif; background-color: #fff; color: #000; padding:20px; font-size:18px; overscroll-behavior:contain;" onload="e=document.getElementById(\'captcha\');if(e){e.focus();} if(solveSimpleChallenge) {solveSimpleChallenge(0,0);}">\n<div style="max-width:400px;">\n<hr noshade="" size="1" style="color:#ccc; background-color:#ccc;"><br>\n<form id="captcha-form" action="index" method="post">\n<noscrip